import dependencies

In [7]:
import tensorflow as tf
import numpy as np
import random
import time
import sys
import string

RNN with num_layers LSTM layers and a fully-connected output layer
The network allows for a dynamic number of iterations, depending on the inputs it receives.

out (fc layer; out_size) <- lstm <- lstm <- in (in_size)

In [2]:
class ModelNetwork:
    def __init__(self, in_size, lstm_size, num_layers, out_size, session, learning_rate=0.003, name="rnn"):
        self.scope = name

        self.in_size = in_size
        self.lstm_size = lstm_size
        self.num_layers = num_layers
        self.out_size = out_size

        self.session = session

        self.learning_rate = tf.constant( learning_rate )

        # Last state of LSTM, used when running the network in TEST mode
        self.lstm_last_state = np.zeros((self.num_layers*2*self.lstm_size,))

        with tf.variable_scope(self.scope):
            ## (batch_size, timesteps, in_size)
            self.xinput = tf.placeholder(tf.float32, shape=(None, None, self.in_size), name="xinput")
            self.lstm_init_value = tf.placeholder(tf.float32, shape=(None, self.num_layers*2*self.lstm_size), name="lstm_init_value")

            # LSTM
            self.lstm_cells = [ tf.contrib.rnn.BasicLSTMCell(self.lstm_size, forget_bias=1.0, state_is_tuple=False) for i in range(self.num_layers)]
            self.lstm = tf.contrib.rnn.MultiRNNCell(self.lstm_cells, state_is_tuple=False)

            # Iteratively compute output of recurrent network
            outputs, self.lstm_new_state = tf.nn.dynamic_rnn(self.lstm, self.xinput, initial_state=self.lstm_init_value, dtype=tf.float32)

            # Linear activation (FC layer on top of the LSTM net)
            self.rnn_out_W = tf.Variable(tf.random_normal( (self.lstm_size, self.out_size), stddev=0.01 ))
            self.rnn_out_B = tf.Variable(tf.random_normal( (self.out_size, ), stddev=0.01 ))

            outputs_reshaped = tf.reshape( outputs, [-1, self.lstm_size] )
            network_output = ( tf.matmul( outputs_reshaped, self.rnn_out_W ) + self.rnn_out_B )

            batch_time_shape = tf.shape(outputs)
            self.final_outputs = tf.reshape( tf.nn.softmax( network_output), (batch_time_shape[0], batch_time_shape[1], self.out_size) )


            ## Training: provide target outputs for supervised training.
            self.y_batch = tf.placeholder(tf.float32, (None, None, self.out_size))
            y_batch_long = tf.reshape(self.y_batch, [-1, self.out_size])

            self.cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=network_output, labels=y_batch_long) )
            self.train_op = tf.train.RMSPropOptimizer(self.learning_rate, 0.9).minimize(self.cost)


    ## Input: X is a single element, not a list!
    def run_step(self, x, init_zero_state=True):
        ## Reset the initial state of the network.
        if init_zero_state:
            init_value = np.zeros((self.num_layers*2*self.lstm_size,))
        else:
            init_value = self.lstm_last_state

        out, next_lstm_state = self.session.run([self.final_outputs, self.lstm_new_state], feed_dict={self.xinput:[x], self.lstm_init_value:[init_value]   } )

        self.lstm_last_state = next_lstm_state[0]

        return out[0][0]


    ## xbatch must be (batch_size, timesteps, input_size)
    ## ybatch must be (batch_size, timesteps, output_size)
    def train_batch(self, xbatch, ybatch):
        init_value = np.zeros((xbatch.shape[0], self.num_layers*2*self.lstm_size))

        cost, _ = self.session.run([self.cost, self.train_op], feed_dict={self.xinput:xbatch, self.y_batch:ybatch, self.lstm_init_value:init_value   } )

        return cost

Embed string to character-arrays -- it generates an array len(data) x len(vocab)

Vocab is a list of elements

In [3]:
def embed_to_vocab(data_, vocab):
    data = np.zeros((len(data_), len(vocab)))

    cnt=0
    for s in data_:
        v = [0.0]*len(vocab)
        v[vocab.index(s)] = 1.0
        data[cnt, :] = v
        cnt += 1

    return data

def decode_embed(array, vocab):
    return vocab[ array.index(1) ]

In [4]:
ckpt_file = ""
TEST_PREFIX = "The " # Prefix to prompt the network in test mode

## Load the data
data_ = ""
with open('data/dostoevski.txt', 'r', encoding="utf-8") as f:
    data_ += f.read()
data_ = data_.lower()

## Convert to 1-hot coding
vocab = list(set(data_))

data = embed_to_vocab(data_, vocab)


in_size = out_size = len(vocab)
lstm_size = 256
num_layers = 2
batch_size = 64 #128
time_steps = 100 #50

NUM_TRAIN_BATCHES = 19500

LEN_TEST_TEXT = 500 # Number of test characters of text to generate after training the network



## Initialize the network
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
sess = tf.InteractiveSession(config=config)

net = ModelNetwork(in_size = in_size,lstm_size = lstm_size,num_layers = num_layers,out_size = out_size,session = sess,learning_rate = 0.003)

sess.run(tf.global_variables_initializer())

saver = tf.train.Saver(tf.global_variables())


train network

In [5]:
if ckpt_file == "":
    last_time = time.time()

    batch = np.zeros((batch_size, time_steps, in_size))
    batch_y = np.zeros((batch_size, time_steps, in_size))

    possible_batch_ids = range(data.shape[0]-time_steps-1)
    for i in range(NUM_TRAIN_BATCHES):
        # Sample time_steps consecutive samples from the dataset text file
        batch_id = random.sample( possible_batch_ids, batch_size )

        for j in range(time_steps):
            ind1 = [k+j for k in batch_id]
            ind2 = [k+j+1 for k in batch_id]

            batch[:, j, :] = data[ind1, :]
            batch_y[:, j, :] = data[ind2, :]


        cst = net.train_batch(batch, batch_y)

        if (i%10) == 0:
            new_time = time.time()
            diff = new_time - last_time
            last_time = new_time

            print("batch: ",i,"   loss: ",cst,"   speed: ",(100.0/diff)," batches / s")

saver.save(sess, "./model.ckpt")

batch:  0    loss:  4.55531    speed:  134.7300470878544  batches / s
batch:  10    loss:  4.55334    speed:  14.81876504756129  batches / s
batch:  20    loss:  4.54968    speed:  15.092368025296343  batches / s
batch:  30    loss:  4.54402    speed:  15.09004296285082  batches / s
batch:  40    loss:  4.53532    speed:  15.092319149260781  batches / s
batch:  50    loss:  4.52302    speed:  14.757310848490205  batches / s
batch:  60    loss:  4.5043    speed:  13.635605271011581  batches / s
batch:  70    loss:  4.47584    speed:  14.512521367135438  batches / s
batch:  80    loss:  4.4039    speed:  14.646799726375427  batches / s
batch:  90    loss:  3.5386    speed:  14.963249939298702  batches / s
batch:  100    loss:  3.34338    speed:  14.254279270369738  batches / s
batch:  110    loss:  3.29796    speed:  14.883958076554281  batches / s
batch:  120    loss:  3.24736    speed:  15.071814289610046  batches / s
batch:  130    loss:  3.23931    speed:  15.094640567379153  batches

batch:  1130    loss:  1.53284    speed:  14.766106025982872  batches / s
batch:  1140    loss:  1.53557    speed:  14.685680244695979  batches / s
batch:  1150    loss:  1.48756    speed:  14.905157439684242  batches / s
batch:  1160    loss:  1.44543    speed:  14.91851892110148  batches / s
batch:  1170    loss:  1.49895    speed:  13.897780913222137  batches / s
batch:  1180    loss:  1.54745    speed:  14.86958556675586  batches / s
batch:  1190    loss:  1.46449    speed:  14.89176964137615  batches / s
batch:  1200    loss:  1.46642    speed:  14.878459705052546  batches / s
batch:  1210    loss:  1.49202    speed:  14.867362362005236  batches / s
batch:  1220    loss:  1.45657    speed:  14.862931113358384  batches / s
batch:  1230    loss:  1.49903    speed:  14.880667224314198  batches / s
batch:  1240    loss:  1.54238    speed:  14.86295692085383  batches / s
batch:  1250    loss:  1.44352    speed:  14.889555655656164  batches / s
batch:  1260    loss:  1.5107    speed:  1

batch:  2250    loss:  1.27174    speed:  14.803459494913913  batches / s
batch:  2260    loss:  1.28686    speed:  14.61467545750828  batches / s
batch:  2270    loss:  1.35364    speed:  15.216770292109613  batches / s
batch:  2280    loss:  1.30509    speed:  15.247022947360179  batches / s
batch:  2290    loss:  1.34347    speed:  15.233036951056611  batches / s
batch:  2300    loss:  1.31424    speed:  15.235385814355713  batches / s
batch:  2310    loss:  1.32746    speed:  15.244672156992022  batches / s
batch:  2320    loss:  1.2964    speed:  15.170494376870991  batches / s
batch:  2330    loss:  1.31666    speed:  15.108429525221215  batches / s
batch:  2340    loss:  1.28756    speed:  15.133655600787659  batches / s
batch:  2350    loss:  1.2654    speed:  15.228381758243762  batches / s
batch:  2360    loss:  1.31874    speed:  15.242361419400718  batches / s
batch:  2370    loss:  1.27517    speed:  15.257510023155612  batches / s
batch:  2380    loss:  1.29712    speed: 

batch:  3370    loss:  1.21979    speed:  14.653283512519684  batches / s
batch:  3380    loss:  1.19229    speed:  14.417048309216604  batches / s
batch:  3390    loss:  1.22541    speed:  14.70189270870872  batches / s
batch:  3400    loss:  1.19338    speed:  13.844943383765237  batches / s
batch:  3410    loss:  1.22944    speed:  13.750074416801754  batches / s
batch:  3420    loss:  1.15342    speed:  13.227303502449002  batches / s
batch:  3430    loss:  1.21378    speed:  11.981989352857209  batches / s
batch:  3440    loss:  1.16012    speed:  10.798362053032283  batches / s
batch:  3450    loss:  1.19597    speed:  12.291586766642325  batches / s
batch:  3460    loss:  1.16301    speed:  12.026892637339653  batches / s
batch:  3470    loss:  1.22725    speed:  13.402662104936516  batches / s
batch:  3480    loss:  1.19416    speed:  13.437066546653805  batches / s
batch:  3490    loss:  1.17664    speed:  13.426947223213093  batches / s
batch:  3500    loss:  1.18071    speed

batch:  4490    loss:  1.11205    speed:  13.913784246816755  batches / s
batch:  4500    loss:  1.10452    speed:  13.683743136194924  batches / s
batch:  4510    loss:  1.122    speed:  13.681871972403897  batches / s
batch:  4520    loss:  1.10597    speed:  13.85194460733409  batches / s
batch:  4530    loss:  1.15605    speed:  13.840411819092596  batches / s
batch:  4540    loss:  1.07068    speed:  13.75691953401993  batches / s
batch:  4550    loss:  1.15215    speed:  13.446497574307754  batches / s
batch:  4560    loss:  1.11409    speed:  14.191648140306452  batches / s
batch:  4570    loss:  1.09937    speed:  14.3628531352668  batches / s
batch:  4580    loss:  1.12732    speed:  14.6855979738845  batches / s
batch:  4590    loss:  1.13363    speed:  14.32874768067219  batches / s
batch:  4600    loss:  1.14589    speed:  13.944219650556054  batches / s
batch:  4610    loss:  1.07469    speed:  12.989265586763182  batches / s
batch:  4620    loss:  1.1554    speed:  13.738

...

batch:  15500    loss:  0.844346    speed:  13.54335398066153  batches / s
batch:  15510    loss:  0.915989    speed:  13.625708008726969  batches / s
batch:  15520    loss:  0.896619    speed:  13.454554121480172  batches / s
batch:  15530    loss:  0.889219    speed:  13.847499893609122  batches / s
batch:  15540    loss:  0.896504    speed:  13.484145494893836  batches / s
batch:  15550    loss:  0.920318    speed:  13.366649592545915  batches / s
batch:  15560    loss:  0.874601    speed:  13.880304781987622  batches / s
batch:  15570    loss:  0.873082    speed:  14.285899670729451  batches / s
batch:  15580    loss:  0.860568    speed:  14.134067379905698  batches / s
batch:  15590    loss:  0.917554    speed:  13.650336657863912  batches / s
batch:  15600    loss:  0.881515    speed:  13.600116237870907  batches / s
batch:  15610    loss:  0.862079    speed:  13.013705876875747  batches / s
batch:  15620    loss:  0.87334    speed:  12.417856459687687  batches / s
batch:  15630 

batch:  16590    loss:  0.880385    speed:  13.411766978069545  batches / s
batch:  16600    loss:  0.89224    speed:  12.377138361596943  batches / s
batch:  16610    loss:  0.880937    speed:  12.829986883711634  batches / s
batch:  16620    loss:  0.892124    speed:  13.895714950301407  batches / s
batch:  16630    loss:  0.914965    speed:  13.624193879081018  batches / s
batch:  16640    loss:  0.863874    speed:  13.19427354666598  batches / s
batch:  16650    loss:  0.873561    speed:  13.48236535423392  batches / s
batch:  16660    loss:  0.869036    speed:  14.147070228974007  batches / s
batch:  16670    loss:  0.894707    speed:  12.579621674154907  batches / s
batch:  16680    loss:  0.873389    speed:  13.777101213689754  batches / s
batch:  16690    loss:  0.851762    speed:  13.046194364377873  batches / s
batch:  16700    loss:  0.852648    speed:  12.911563585072617  batches / s
batch:  16710    loss:  0.849316    speed:  12.53954948550615  batches / s
batch:  16720   

batch:  17680    loss:  0.90771    speed:  14.386916689085282  batches / s
batch:  17690    loss:  0.865828    speed:  13.198099850517336  batches / s
batch:  17700    loss:  0.875365    speed:  13.968758541464906  batches / s
batch:  17710    loss:  0.876805    speed:  13.663417664997347  batches / s
batch:  17720    loss:  0.880402    speed:  13.780436779271572  batches / s
batch:  17730    loss:  0.896511    speed:  13.517097534549247  batches / s
batch:  17740    loss:  0.884832    speed:  14.060251756809064  batches / s
batch:  17750    loss:  0.873253    speed:  13.205516722690017  batches / s
batch:  17760    loss:  0.893375    speed:  13.979805944093295  batches / s
batch:  17770    loss:  0.869144    speed:  14.184345452584619  batches / s
batch:  17780    loss:  0.86094    speed:  14.100086211374018  batches / s
batch:  17790    loss:  0.850895    speed:  14.119039653077857  batches / s
batch:  17800    loss:  0.888284    speed:  13.116319941209122  batches / s
batch:  17810 

batch:  18770    loss:  0.869546    speed:  13.018169552068997  batches / s
batch:  18780    loss:  0.841727    speed:  12.335285219855523  batches / s
batch:  18790    loss:  0.896316    speed:  13.807245012675311  batches / s
batch:  18800    loss:  0.921412    speed:  14.3557987353369  batches / s
batch:  18810    loss:  0.846896    speed:  14.351695134330718  batches / s
batch:  18820    loss:  0.853943    speed:  14.407715600353717  batches / s
batch:  18830    loss:  0.880274    speed:  14.06035639313836  batches / s
batch:  18840    loss:  0.845319    speed:  13.501540332332713  batches / s
batch:  18850    loss:  0.870728    speed:  13.228469931145348  batches / s
batch:  18860    loss:  0.861649    speed:  13.878959952903637  batches / s
batch:  18870    loss:  0.852794    speed:  13.906090562138992  batches / s
batch:  18880    loss:  0.886577    speed:  13.884788049539875  batches / s
batch:  18890    loss:  0.875655    speed:  13.99270050321697  batches / s
batch:  18900   

'./model.ckpt'

generate text

In [37]:
TEST_PREFIX = "прохожу и играюсь "

for i in range(len(TEST_PREFIX)):
    out = net.run_step( embed_to_vocab(TEST_PREFIX[i], vocab) , i==0)

print("SENTENCE:")
gen_str = TEST_PREFIX
for i in range(LEN_TEST_TEXT):
    element = np.random.choice( range(len(vocab)), p=out ) # Sample character from the network according to the generated output probabilities
    gen_str += vocab[element]

    out = net.run_step( embed_to_vocab(vocab[element], vocab) , False )
print(gen_str)

SENTENCE:
прохожу и играюсь несколько времени, одечной дымя через-плуту, и не окружают в себе не за пер-с; отчего косы ты пройду, чтобы быть послучше индцев после самым веселым и прочее, и сердце его собственного и прыснула. канкалыва желтые, ни за что не хотел вы, будеть, всё равно. он относит к петца и пошли и кого ободрилась!.. да и зачем же, дмитрий прокофьич? слушал он вполне. разве это возможно? слышите! фру, он жениться родиом родятся, — продолжал разумихин, — но не стыдно со всеми экономи средства и в самых двадцат


In [5]:
text = data_
valid_size = 2000
valid_text = text[:valid_size]
train_text = text[valid_size:]
train_size = len(train_text)
print(train_size, train_text[:64])
print(valid_size, valid_text[:64])

1091216 ма… любопытно, чего люди больше всего боятся? нового шага, новог
2000 
в начале июля, в чрезвычайно жаркое время, под вечер, один моло


In [43]:
alphabet = 'абвгдеёжзийклмнопрстуфхцчшщьыъэюя!.…,—1234567890?»;-\n«{}[]():–„“';
vocabulary_size = len(alphabet) + 1 # [a-z] + ' '

def char2id(char):
  if char in alphabet:
    return  alphabet.index(char) + 1
  elif char == ' ':
    return 0
  else:
    print('Unexpected character: %s' % char)
    return 0
  
def id2char(dictid):
  if dictid > 0:
    return alphabet[dictid - 1]
  else:
    return ' '

print(char2id('а'), char2id('я'), char2id(' '), char2id('ï'))
print(id2char(1), id2char(32), id2char(0))

Unexpected character: ï
1 33 0 0
а ю  


In [44]:
batch_size=64
num_unrollings=10

class BatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, vocabulary_size), dtype=np.float)
    for b in range(self._batch_size):
      batch[b, char2id(self._text[self._cursor[b]])] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c) for c in np.argmax(probabilities, 1)]

def batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, characters(b))]
  return s

train_batches = BatchGenerator(train_text, batch_size, num_unrollings)
valid_batches = BatchGenerator(valid_text, 1, 1)

print(batches2string(train_batches.next()))
print(batches2string(train_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))
print(batches2string(valid_batches.next()))

['ма… любопыт', 'нность, — п', 'идания… бер', 'х, крашеный', 'нь расчетли', 'лько я знал', 'кже и всех ', 'ками на мик', 'н, но засну', 'я.\n\n— госпо', 'то она на о', 'о писцы, од', 'ь его серде', 'еня! теперь', 'и расписалс', 'япах и фура', 'ерьги взял?', 'он снова, с', 'тия, да и д', 'ончил бы, и', 'мод; комнат', 'ыстрей, но ', 'то действит', 'сь!.. на вс', 'в, жирных к', 'щал, в само', 'а, совсем п', 'одя, настоя', 'льно и вдум', 'не нравилис', 'о практичес', ' уже до вхо', 'ерся на тро', ', марфа пет', 'ах, нет, пе', 'следнего мг', 'ал, что ты ', 'зала ее, и ', 'йным и подо', 'рывает,[55 ', 'скрыть. не ', 'провалился?', 'о молодого ', 'ачал петр п', 'мнате стоит', 'ением говор', 'ко страдающ', 'визгом, как', 'о и коротко', 'опроса.\n\n— ', 'на устройст', 'го отвращен', 'ло… совсем ', 'оситесь. пр', 'ать глотков', 'оставьте вс', 'у публику, ', 'ы, никакого', 'арева. весь', 'лся по лест', 'нула она, к', 'се-таки шел', 'исле их оши', 'он часто и ']
['тно, чего л', 'пожалуй, бы', 'ру 

In [45]:
def logprob(predictions, labels):
  """Log-probability of the true labels in a predicted batch."""
  predictions[predictions < 1e-10] = 1e-10
  return np.sum(np.multiply(labels, -np.log(predictions))) / labels.shape[0]

def sample_distribution(distribution):
  """Sample one element from a distribution assumed to be an array of normalized
  probabilities.
  """
  r = random.uniform(0, 1)
  s = 0
  for i in range(len(distribution)):
    s += distribution[i]
    if s >= r:
      return i
  return len(distribution) - 1

def sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, vocabulary_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, vocabulary_size])
  return b/np.sum(b, 1)[:,None]

In [46]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # Input gate: input, previous output, and bias.
  ix = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  im = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ib = tf.Variable(tf.zeros([1, num_nodes]))
  # Forget gate: input, previous output, and bias.
  fx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  fm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  fb = tf.Variable(tf.zeros([1, num_nodes]))
  # Memory cell: input, state and bias.                             
  cx = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  cm = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  cb = tf.Variable(tf.zeros([1, num_nodes]))
  # Output gate: input, previous output, and bias.
  ox = tf.Variable(tf.truncated_normal([vocabulary_size, num_nodes], -0.1, 0.1))
  om = tf.Variable(tf.truncated_normal([num_nodes, num_nodes], -0.1, 0.1))
  ob = tf.Variable(tf.zeros([1, num_nodes]))
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, vocabulary_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([vocabulary_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    input_gate = tf.sigmoid(tf.matmul(i, ix) + tf.matmul(o, im) + ib)
    forget_gate = tf.sigmoid(tf.matmul(i, fx) + tf.matmul(o, fm) + fb)
    update = tf.matmul(i, cx) + tf.matmul(o, cm) + cb
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(tf.matmul(i, ox) + tf.matmul(o, om) + ob)
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,vocabulary_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs, 0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        labels=tf.concat(train_labels, 0), logits=logits))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, vocabulary_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [47]:
num_steps = 7001
summary_frequency = 100

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print('Initialized')
  mean_loss = 0
  for step in range(num_steps):
    batches = train_batches.next()
    feed_dict = dict()
    for i in range(num_unrollings + 1):
      feed_dict[train_data[i]] = batches[i]
    _, l, predictions, lr = session.run(
      [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
    mean_loss += l
    if step % summary_frequency == 0:
      if step > 0:
        mean_loss = mean_loss / summary_frequency
      # The mean loss is an estimate of the loss over the last few batches.
      print(
        'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
      mean_loss = 0
      labels = np.concatenate(list(batches)[1:])
      print('Minibatch perplexity: %.2f' % float(
        np.exp(logprob(predictions, labels))))
      if step % (summary_frequency * 10) == 0:
        # Generate some samples.
        print('=' * 80)
        for _ in range(5):
          feed = sample(random_distribution())
          sentence = characters(feed)[0]
          reset_sample_state.run()
          for _ in range(79):
            prediction = sample_prediction.eval({sample_input: feed})
            feed = sample(prediction)
            sentence += characters(feed)[0]
          print(sentence)
        print('=' * 80)
      # Measure validation set perplexity.
      reset_sample_state.run()
      valid_logprob = 0
      for _ in range(valid_size):
        b = valid_batches.next()
        predictions = sample_prediction.eval({sample_input: b[0]})
        valid_logprob = valid_logprob + logprob(predictions, b[1])
      print('Validation set perplexity: %.2f' % float(np.exp(
        valid_logprob / valid_size)))

Initialized
Average loss at step 0: 4.124578 learning rate: 10.000000
Minibatch perplexity: 61.84
гевъ![рч! {д}9дтфняёъ3;а9бщомр}н 6[яэж1?.щ 1
а7 {4(ъ]тд} эмм ьч?«си нн;о—сп?гэс 
}»п д3.,чи8са}й  п«0в5тяй:д—]6с  ешзыл б.юфя»ъ2.х2!нкт э 2яалл!б{в— оахещ»—[. уи
р5 4…э ! {« нлнл—[:й
.онд9у…ущ2а5в3;у]ъ0й-ю1(е ).хуч{е«:б3эбе—дло9тё2ьм.ыкуй:фхм
леахв…[с[»18узл-цьпчв-унще?щ- , 45цсйбдвы,б3,мн6ьцз!я (м 6а4
с{эвёф«ьо ь][ьнеё
ф
9у8м )о9 л 3 бнт.(б»9(ешщм,-»й»«шбё9]фи,уя8»7п6…оиь43з]0шх…е8?я5ес…
и—}ге оъо5оу
Validation set perplexity: 39.28
Unexpected character: c
Unexpected character: i
Unexpected character: n
Unexpected character: q
Unexpected character: s
Unexpected character: o
Unexpected character: u
Unexpected character: s
Unexpected character: m
Unexpected character: a
Unexpected character: l
Unexpected character: b
Unexpected character: r
Unexpected character: o
Unexpected character: u
Unexpected character: g
Unexpected character: h
Unexpected character: s
Unexpected character: '
Unexpect

Validation set perplexity: 10.04
Unexpected character: v
Unexpected character: t
Unexpected character: o
Unexpected character: u
Unexpected character: t
Unexpected character: c
Unexpected character: o
Unexpected character: u
Unexpected character: r
Unexpected character: t
Unexpected character: „
Unexpected character: “
Unexpected character: „
Unexpected character: “
Unexpected character: b
Unexpected character: o
Unexpected character: n
Unexpected character: n
Unexpected character: e
Unexpected character: g
Unexpected character: u
Unexpected character: e
Unexpected character: r
Unexpected character: r
Unexpected character: e
Unexpected character: „
Unexpected character: “
Unexpected character: a
Unexpected character: v
Unexpected character: №
Unexpected character: &
Unexpected character: #
Average loss at step 400: 2.245984 learning rate: 10.000000
Minibatch perplexity: 9.67
Validation set perplexity: 9.28
Unexpected character: a
Unexpected character: s
Unexpected character: s
Unexpect

Unexpected character: u
Unexpected character: –
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: v
Average loss at step 1000: 1.999322 learning rate: 10.000000
Minibatch perplexity: 6.47
ной сказал еще бметь. догу, — петрой. это взох? то косек, итара отчетно, и остра
шил присть смотрик он вы него. на уна вы вы тас не он вотра, ведьи; вот суждают 


— вышию! дя могла узаменные, раз шевацом обоевка роготном испоэтал он и повадн
дя а мину был не какии! все нахидал ответне, что, что всем и на  вольбо за дащей
0ного еще-то я очень и тою ую стеря» трообо во весь, он даждо веприв а сювища и 
Validation set perplexity: 7.49
Unexpected character: c
Unexpected character: h
Unexpected character: e
Unexpected character: r
Unexpected character: a
Unexpected character: m
Unexpected character: i
Unexpected character: v
Unexpected character: i
Unexpected character: i
Unexpected characte

Validation set perplexity: 6.82
Unexpected character: v
Unexpected character: i
Unexpected character: i
Unexpected character: e
Unexpected character: n
Unexpected character: t
Unexpected character: o
Unexpected character: u
Unexpected character: t
Unexpected character: e
Unexpected character: s
Unexpected character: l
Unexpected character: e
Unexpected character: t
Unexpected character: t
Unexpected character: r
Unexpected character: e
Unexpected character: s
Unexpected character: „
Unexpected character: “
Unexpected character: c
Unexpected character: i
Unexpected character: n
Unexpected character: q
Unexpected character: s
Unexpected character: o
Unexpected character: u
Unexpected character: s
Unexpected character: v
Unexpected character: h
Unexpected character: o
Unexpected character: n
Unexpected character: n
Unexpected character: e
Unexpected character: u
Unexpected character: r
Unexpected character: –
Unexpected character: m
Unexpected character: a
Unexpected character: l
Unexpect

Unexpected character: i
Unexpected character: v
Average loss at step 1900: 1.849638 learning rate: 10.000000
Minibatch perplexity: 5.57
Validation set perplexity: 6.40
Unexpected character: „
Unexpected character: “
Unexpected character: e
Unexpected character: t
Unexpected character: n
Unexpected character: i
Unexpected character: h
Unexpected character: i
Unexpected character: l
Unexpected character: h
Unexpected character: u
Unexpected character: m
Unexpected character: a
Unexpected character: n
Unexpected character: u
Unexpected character: m
Unexpected character: e
Unexpected character: t
Unexpected character: n
Unexpected character: i
Unexpected character: h
Unexpected character: i
Unexpected character: l
Unexpected character: h
Unexpected character: u
Unexpected character: m
Unexpected character: a
Unexpected character: n
Unexpected character: u
Unexpected character: m
Unexpected character: u
Unexpected character: n
Unexpected character: e
Unexpected character: t
Unexpected chara

...

Unexpected character: 	
Unexpected character: 	
Unexpected character: v
Unexpected character: i
Unexpected character: o
Unexpected character: –
Unexpected character: r
Unexpected character: a
Unexpected character: i
Unexpected character: s
Unexpected character: o
Unexpected character: n
Unexpected character: f
Unexpected character: u
Unexpected character: n
Unexpected character: &
Unexpected character: #
Unexpected character: b
Unexpected character: r
Unexpected character: e
Unexpected character: „
Unexpected character: “
Unexpected character: i
Unexpected character: „
Unexpected character: “
Unexpected character: „
Unexpected character: “
Average loss at step 5000: 1.713335 learning rate: 1.000000
Minibatch perplexity: 5.59
: может. — слышили за напрежущенного упичк, чтобы на слако! вы твердить. как бы 
фина испонятою, пожалую как был отровить… созложении ниекогода, она сказалие вер
. —  плечки; со двар;: как было бых он вот же эте сеходней сама особенно расколя
? — как?-золо и он осу

Unexpected character: a
Unexpected character: –
Unexpected character: –
Unexpected character: „
Unexpected character: “
Unexpected character: a
Unexpected character: s
Unexpected character: s
Unexpected character: e
Unexpected character: z
Unexpected character: c
Unexpected character: a
Unexpected character: u
Unexpected character: s
Unexpected character: &
Unexpected character: #
Unexpected character: i
Unexpected character: v
Average loss at step 5300: 1.690048 learning rate: 1.000000
Minibatch perplexity: 5.30
Validation set perplexity: 5.48
Unexpected character: i
Unexpected character: v
Unexpected character: „
Unexpected character: “
Unexpected character: e
Unexpected character: t
Unexpected character: n
Unexpected character: i
Unexpected character: h
Unexpected character: i
Unexpected character: l
Unexpected character: h
Unexpected character: u
Unexpected character: m
Unexpected character: a
Unexpected character: n
Unexpected character: u
Unexpected character: m
Unexpected charac

Unexpected character: m
Unexpected character: a
Unexpected character: d
Unexpected character: a
Unexpected character: m
Unexpected character: e
Unexpected character: d
Unexpected character: e
Unexpected character: i
Unexpected character: i
Unexpected character: m
Unexpected character: a
Unexpected character: d
Unexpected character: a
Unexpected character: m
Unexpected character: e
Unexpected character: v
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: –
Unexpected character: v
Unexpected character: i
Unexpected character: 	
Unexpected character: 	
Unexpected character: d
Unexpected character: u
Unexpected character: h
Unexpected character: a
Unexpected character: s
Unexpected character: t
Unexpected character: d
Unexpected character: i
Unexpected character: e
Unexpected character: s
Unexpected character: c
Unexpected character: h
Unexpected character: &
Unexpected chara

Unexpected character: „
Unexpected character: “
Unexpected character: „
Unexpected character: “
Average loss at step 6400: 1.669432 learning rate: 1.000000
Minibatch perplexity: 4.65
Validation set perplexity: 5.43
Unexpected character: i
Unexpected character: v
Unexpected character: „
Unexpected character: “
Unexpected character: v
Unexpected character: l
Unexpected character: a
Unexpected character: n
Unexpected character: a
Unexpected character: t
Unexpected character: u
Unexpected character: r
Unexpected character: e
Unexpected character: e
Unexpected character: t
Unexpected character: l
Unexpected character: a
Unexpected character: v
Unexpected character: &
Unexpected character: #
Unexpected character: r
Unexpected character: i
Unexpected character: t
Unexpected character: &
Unexpected character: #
Unexpected character: x
Unexpected character: a
Unexpected character: x
Unexpected character: a
Unexpected character: t
Unexpected character: e
Unexpected character: n
Unexpected charac

Unexpected character: –
Unexpected character: x
Unexpected character: i
Unexpected character: x
Unexpected character: i
Unexpected character: v
Unexpected character: i
Unexpected character: i
Unexpected character: –
Unexpected character: &
Unexpected character: #
Unexpected character: x
Unexpected character: i
Unexpected character: x
Unexpected character: i
Unexpected character: 	
Unexpected character: 	
Unexpected character: c
Unexpected character: i
Unexpected character: n
Unexpected character: q
Unexpected character: s
Unexpected character: o
Unexpected character: u
Unexpected character: s
Unexpected character: c
Unexpected character: i
Unexpected character: n
Unexpected character: q
Unexpected character: s
Unexpected character: o
Unexpected character: u
Unexpected character: s
Average loss at step 6900: 1.696227 learning rate: 1.000000
Minibatch perplexity: 5.30
Validation set perplexity: 5.43
Unexpected character: v
Unexpected character: c
Unexpected character: r
Unexpected charac

In [48]:
batch_size=64
num_unrollings=10
bigram_size = vocabulary_size * vocabulary_size

class BigramBatchGenerator(object):
  def __init__(self, text, batch_size, num_unrollings):
    self._text = text
    self._text_size = len(text)
    self._batch_size = batch_size
    self._num_unrollings = num_unrollings
    segment = self._text_size // batch_size
    self._cursor = [ offset * segment for offset in range(batch_size)]
    self._last_batch = self._next_batch()
  
  def _next_batch(self):
    """Generate a single batch from the current cursor position in the data."""
    batch = np.zeros(shape=(self._batch_size, bigram_size), dtype=np.float)
    for b in range(self._batch_size):
      first_char = self._text[self._cursor[b]]
      if self._cursor[b] + 1 == self._text_size:
        second_char = ' '
      else:
        second_char = self._text[self._cursor[b] + 1]
      batch[b, char2id(first_char) * vocabulary_size + char2id(second_char)] = 1.0
      self._cursor[b] = (self._cursor[b] + 1) % self._text_size
    return batch
  
  def next(self):
    """Generate the next array of batches from the data. The array consists of
    the last batch of the previous array, followed by num_unrollings new ones.
    """
    batches = [self._last_batch]
    for step in range(self._num_unrollings):
      batches.append(self._next_batch())
    self._last_batch = batches[-1]
    return batches

def bigram_characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return ['({0},{1})'.format(id2char(c//vocabulary_size), id2char(c % vocabulary_size))
          for c in np.argmax(probabilities, 1)]

def bigram_first_characters(probabilities):
  """Turn a 1-hot encoding or a probability distribution over the possible
  characters back into its (most likely) character representation."""
  return [id2char(c//vocabulary_size)
          for c in np.argmax(probabilities, 1)]

def bigram_batches2string(batches):
  """Convert a sequence of batches back into their (most likely) string
  representation."""
  s = [''] * batches[0].shape[0]
  for b in batches:
    s = [''.join(x) for x in zip(s, bigram_characters(b))]
  return s

bigram_train_batches = BigramBatchGenerator(train_text, batch_size, num_unrollings)
bigram_valid_batches = BigramBatchGenerator(valid_text, 1, 1)

# output each bigram instead of a single char
print(bigram_batches2string(bigram_train_batches.next()))
print(bigram_batches2string(bigram_train_batches.next()))
print(bigram_batches2string(bigram_valid_batches.next()))
print(bigram_batches2string(bigram_valid_batches.next()))

['(м,а)(а,…)(…, )( ,л)(л,ю)(ю,б)(б,о)(о,п)(п,ы)(ы,т)(т,н)', '(н,н)(н,о)(о,с)(с,т)(т,ь)(ь,,)(,, )( ,—)(—, )( ,п)(п,о)', '(и,д)(д,а)(а,н)(н,и)(и,я)(я,…)(…, )( ,б)(б,е)(е,р)(р,у)', '(х,,)(,, )( ,к)(к,р)(р,а)(а,ш)(ш,е)(е,н)(н,ы)(ы,й)(й, )', '(н,ь)(ь, )( ,р)(р,а)(а,с)(с,ч)(ч,е)(е,т)(т,л)(л,и)(и,в)', '(л,ь)(ь,к)(к,о)(о, )( ,я)(я, )( ,з)(з,н)(н,а)(а,л)(л,,)', '(к,ж)(ж,е)(е, )( ,и)(и, )( ,в)(в,с)(с,е)(е,х)(х, )( ,ф)', '(к,а)(а,м)(м,и)(и, )( ,н)(н,а)(а, )( ,м)(м,и)(и,к)(к,о)', '(н,,)(,, )( ,н)(н,о)(о, )( ,з)(з,а)(а,с)(с,н)(н,у)(у,т)', '(я,.)(.,\n)(\n,\n)(\n,—)(—, )( ,г)(г,о)(о,с)(с,п)(п,о)(о,д)', '(т,о)(о, )( ,о)(о,н)(н,а)(а, )( ,н)(н,а)(а, )( ,о)(о,д)', '(о, )( ,п)(п,и)(и,с)(с,ц)(ц,ы)(ы,,)(,, )( ,о)(о,д)(д,е)', '(ь, )( ,е)(е,г)(г,о)(о, )( ,с)(с,е)(е,р)(р,д)(д,е)(е,ч)', '(е,н)(н,я)(я,!)(!, )( ,т)(т,е)(е,п)(п,е)(е,р)(р,ь)(ь, )', '(и, )( ,р)(р,а)(а,с)(с,п)(п,и)(и,с)(с,а)(а,л)(л,с)(с,я)', '(я,п)(п,а)(а,х)(х, )( ,и)(и, )( ,ф)(ф,у)(у,р)(р,а)(а,ж)', '(е,р)(р,ь)(ь,г)(г,и)(и, )( ,в)(в,з)(з,я)(я,л)(л,?)

In [49]:
num_nodes = 64

graph = tf.Graph()
with graph.as_default():
  
  # Parameters:
  # big matrix
  ifcox = tf.Variable(tf.truncated_normal([bigram_size, 4 * num_nodes], -0.1, 0.1))
  ifcom = tf.Variable(tf.truncated_normal([num_nodes, 4 * num_nodes], -0.1, 0.1))
  ifcob = tf.Variable(tf.zeros([1, 4 * num_nodes]))
  
  # Variables saving state across unrollings.
  saved_output = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  saved_state = tf.Variable(tf.zeros([batch_size, num_nodes]), trainable=False)
  # Classifier weights and biases.
  w = tf.Variable(tf.truncated_normal([num_nodes, bigram_size], -0.1, 0.1))
  b = tf.Variable(tf.zeros([bigram_size]))
  
  # Definition of the cell computation.
  def lstm_cell(i, o, state):
    """Create a LSTM cell. See e.g.: http://arxiv.org/pdf/1402.1128v1.pdf
    Note that in this formulation, we omit the various connections between the
    previous state and the gates."""
    all_gates = tf.matmul(i, ifcox) + tf.matmul(o, ifcom) + ifcob
    input_gate = tf.sigmoid(all_gates[:, 0:num_nodes])
    forget_gate = tf.sigmoid(all_gates[:, num_nodes:2*num_nodes])
    update = all_gates[:, 2*num_nodes:3*num_nodes]
    state = forget_gate * state + input_gate * tf.tanh(update)
    output_gate = tf.sigmoid(all_gates[:, 3*num_nodes:])
        
    return output_gate * tf.tanh(state), state

  # Input data.
  train_data = list()
  for _ in range(num_unrollings + 1):
    train_data.append(
      tf.placeholder(tf.float32, shape=[batch_size,bigram_size]))
  train_inputs = train_data[:num_unrollings]
  train_labels = train_data[1:]  # labels are inputs shifted by one time step.

  # Unrolled LSTM loop.
  outputs = list()
  output = saved_output
  state = saved_state
  for i in train_inputs:
    output, state = lstm_cell(i, output, state)
    outputs.append(output)

  # State saving across unrollings.
  with tf.control_dependencies([saved_output.assign(output),
                                saved_state.assign(state)]):
    # Classifier.
    logits = tf.nn.xw_plus_b(tf.concat(outputs,0), w, b)
    loss = tf.reduce_mean(
      tf.nn.softmax_cross_entropy_with_logits(
        logits=logits, labels=tf.concat(train_labels,0)))

  # Optimizer.
  global_step = tf.Variable(0)
  learning_rate = tf.train.exponential_decay(
    10.0, global_step, 5000, 0.1, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate)
  gradients, v = zip(*optimizer.compute_gradients(loss))
  gradients, _ = tf.clip_by_global_norm(gradients, 1.25)
  optimizer = optimizer.apply_gradients(
    zip(gradients, v), global_step=global_step)

  # Predictions.
  train_prediction = tf.nn.softmax(logits)
  
  # Sampling and validation eval: batch 1, no unrolling.
  sample_input = tf.placeholder(tf.float32, shape=[1, bigram_size])
  saved_sample_output = tf.Variable(tf.zeros([1, num_nodes]))
  saved_sample_state = tf.Variable(tf.zeros([1, num_nodes]))
  reset_sample_state = tf.group(
    saved_sample_output.assign(tf.zeros([1, num_nodes])),
    saved_sample_state.assign(tf.zeros([1, num_nodes])))
  sample_output, sample_state = lstm_cell(
    sample_input, saved_sample_output, saved_sample_state)
  with tf.control_dependencies([saved_sample_output.assign(sample_output),
                                saved_sample_state.assign(sample_state)]):
    sample_prediction = tf.nn.softmax(tf.nn.xw_plus_b(sample_output, w, b))

In [50]:
def bigram_sample(prediction):
  """Turn a (column) prediction into 1-hot encoded samples."""
  p = np.zeros(shape=[1, bigram_size], dtype=np.float)
  p[0, sample_distribution(prediction[0])] = 1.0
  return p

def bigram_random_distribution():
  """Generate a random column of probabilities."""
  b = np.random.uniform(0.0, 1.0, size=[1, bigram_size])
  return b/np.sum(b, 1)[:,None]

In [ ]:
num_steps = 7001
summary_frequency = 100

def exec_graph_bigram(graph):
  with tf.Session(graph=graph) as session:
    tf.initialize_all_variables().run()
    print('Initialized')
    mean_loss = 0
    for step in range(num_steps):
      batches = bigram_train_batches.next()
      feed_dict = dict()
      for i in range(num_unrollings + 1):
        feed_dict[train_data[i]] = batches[i]
      _, l, predictions, lr = session.run(
        [optimizer, loss, train_prediction, learning_rate], feed_dict=feed_dict)
      mean_loss += l
      if step % summary_frequency == 0:
        if step > 0:
          mean_loss = mean_loss / summary_frequency
        # The mean loss is an estimate of the loss over the last few batches.
        print(
          'Average loss at step %d: %f learning rate: %f' % (step, mean_loss, lr))
        mean_loss = 0
        labels = np.concatenate(list(batches)[1:])
        print('Minibatch perplexity: %.2f' % float(
          np.exp(logprob(predictions, labels))))
        if step % (summary_frequency * 10) == 0:
          # Generate some samples.
          print('=' * 80)
          for _ in range(5):
            feed = bigram_sample(bigram_random_distribution())
            bigram_sentence = bigram_characters(feed)[0]
            sentence = bigram_first_characters(feed)[0]
            reset_sample_state.run()
            for _ in range(79):
              prediction = sample_prediction.eval({sample_input: feed})
              feed = bigram_sample(prediction)
              bigram_sentence += bigram_characters(feed)[0]
              sentence += bigram_first_characters(feed)[0]
            print('bigrams:', bigram_sentence)
            print('chars:', sentence)
          print('=' * 80)
        # Measure validation set perplexity.
        reset_sample_state.run()
        valid_logprob = 0
        for _ in range(valid_size):
          b = bigram_valid_batches.next()
          predictions = sample_prediction.eval({sample_input: b[0]})
          valid_logprob = valid_logprob + logprob(predictions, b[1])
        print('Validation set perplexity: %.2f' % float(np.exp(
          valid_logprob / valid_size)))
        
%time exec_graph_bigram(graph)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Average loss at step 0: 8.252423 learning rate: 10.000000
Minibatch perplexity: 3836.91
bigrams: (м,ш)(в,,)(д,б)(с,ж)(-,])((,3)(м,1)({,п)(ц,ы)(],8)(ш,д)([,ш)(ы,.)(ъ,ч)(з,ж)(п,5)((,х)(к,.)(7,9)(—,с)(з,щ)(5,ж)((,2)(х,й)(д,:)(у,м)(к,в)([,у)(о,2)(;, )(э, )(-,э)(!,9)(д,ф)(к,2)(е,х)((,4)((,у)( ,2)(.,))(ж,ь)(щ,»)(ц,-)(ш,х)(л,ы)(ю,ы)(с,5)(5,п)(ж,-)(ж,ю)(5,-)(р,3)(е,-)(,,«)([,3)(0,;)(р,—)(6,—)(м,й)(9,])(ь,я)(9, )(:,
)(},р)(ё,.)(9,7)(ш,«)(о,9)(с, )(о,л)(.,;)(е,3)(м,а)(ж,ь)(т,и)(г,:)(ш,{)(р,е)(э,«)(«,к)
chars: мвдс-(м{ц]ш[ыъзп(к7—з5(хдук[о;э-!дке(( .жщцшлюс5жж5ре,[0р6м9ь9:}ё9шосо.емжтгшрэ«
bigrams: (н,я)(6,б)(з,:)(г,ф)(у,ы)(?,.)(ж,ч)(:,ц)(9,3)(6,м)(с,2)(о,9)(т,3)(;,н)(х,в)(;, )(«,?)(щ,ъ)(й,!)(а,ъ)(…,м)((,))(5,д)(к,ф)( ,ц)(д,—)(н,4)(ш,))(),х)(у,])(з,»)(я,ч)(э,м)(р,
)(],п)(1,х)(6,и)(в,ц)(т,а)(д,])(щ,г)(г,т)(-,ь)(ъ,ъ)(п,[)(
,8)(?,»)(},,)(ч,ъ)(ф,7)(ч,.)(п,п)(ч,))(?,
)(н,у)(ь,п)(ш,])(щ,ь)(:,ч)(.,))(ж,})(),:)(ё,ю)(п,т

Unexpected character: v
Unexpected character: v
Unexpected character: a
Unexpected character: a
Unexpected character: t
Unexpected character: c
Unexpected character: t
Unexpected character: c
Unexpected character: o
Unexpected character: e
Unexpected character: o
Unexpected character: n
Unexpected character: e
Unexpected character: l
Unexpected character: n
Unexpected character: f
Unexpected character: l
Unexpected character: l
Unexpected character: f
Unexpected character: e
Unexpected character: l
Unexpected character: e
Unexpected character: e
Unexpected character: s
Unexpected character: e
Unexpected character: s
Unexpected character: s
Unexpected character: l
Unexpected character: s
Unexpected character: i
Unexpected character: l
Unexpected character: a
Unexpected character: i
Unexpected character: o
Unexpected character: a
Unexpected character: o
Unexpected character: n
Unexpected character: v
Unexpected character: n
Unexpected character: s
Unexpected character: v
Unexpected chara

Unexpected character: p
Unexpected character: p
Unexpected character: l
Unexpected character: l
Unexpected character: a
Unexpected character: a
Unexpected character: i
Unexpected character: i
Unexpected character: s
Unexpected character: s
Unexpected character: i
Unexpected character: i
Unexpected character: r
Unexpected character: r
Unexpected character: „
Unexpected character: „
Unexpected character: “
Unexpected character: “
Average loss at step 300: 5.130433 learning rate: 10.000000
Minibatch perplexity: 101.33
Validation set perplexity: 109.59
Unexpected character: v
Unexpected character: v
Unexpected character: t
Unexpected character: t
Unexpected character: o
Unexpected character: o
Unexpected character: u
Unexpected character: u
Unexpected character: t
Unexpected character: t
Unexpected character: c
Unexpected character: c
Unexpected character: o
Unexpected character: o
Unexpected character: u
Unexpected character: u
Unexpected character: r
Unexpected character: r
Unexpected ch

Unexpected character: d
Unexpected character: d
Unexpected character: u
Unexpected character: u
Unexpected character: h
Unexpected character: h
Unexpected character: a
Unexpected character: a
Unexpected character: s
Unexpected character: s
Unexpected character: t
Unexpected character: t
Unexpected character: d
Unexpected character: d
Unexpected character: i
Unexpected character: i
Unexpected character: a
Unexpected character: a
Unexpected character: m
Unexpected character: m
Unexpected character: a
Unexpected character: a
Unexpected character: n
Unexpected character: n
Unexpected character: t
Unexpected character: t
Unexpected character: e
Unexpected character: e
Unexpected character: n
Unexpected character: n
Unexpected character: u
Unexpected character: u
Unexpected character: n
Unexpected character: n
Unexpected character: d
Unexpected character: d
Unexpected character: p
Unexpected character: p
Unexpected character: e
Unexpected character: e
Unexpected character: r
Unexpected chara

Unexpected character: –
Unexpected character: –
Unexpected character: a
Unexpected character: a
Unexpected character: p
Unexpected character: p
Unexpected character: p
Unexpected character: p
Unexpected character: o
Unexpected character: o
Unexpected character: r
Unexpected character: r
Unexpected character: t
Unexpected character: t
Unexpected character: e
Unexpected character: e
Unexpected character: z
Unexpected character: z
Unexpected character: s
Unexpected character: s
Unexpected character: a
Unexpected character: a
Unexpected character: p
Unexpected character: p
Unexpected character: i
Unexpected character: i
Unexpected character: e
Unexpected character: e
Unexpected character: r
Unexpected character: r
Unexpected character: r
Unexpected character: r
Unexpected character: e
Unexpected character: e
Unexpected character: &
Unexpected character: &
Unexpected character: #
Unexpected character: #
Unexpected character: l
Unexpected character: l
Unexpected character: '
Unexpected chara

...

Unexpected character: n
Unexpected character: o
Unexpected character: o
Unexpected character: t
Unexpected character: t
Unexpected character: r
Unexpected character: r
Unexpected character: e
Unexpected character: e
Unexpected character: m
Unexpected character: m
Unexpected character: &
Unexpected character: &
Unexpected character: #
Unexpected character: #
Unexpected character: n
Unexpected character: n
Unexpected character: a
Unexpected character: a
Unexpected character: g
Unexpected character: g
Unexpected character: e
Unexpected character: e
Unexpected character: c
Unexpected character: c
Unexpected character: i
Unexpected character: i
Unexpected character: n
Unexpected character: n
Unexpected character: q
Unexpected character: q
Unexpected character: s
Unexpected character: s
Unexpected character: o
Unexpected character: o
Unexpected character: u
Unexpected character: u
Unexpected character: s
Unexpected character: s
Unexpected character: m
Unexpected character: m
Unexpected chara

Unexpected character: c
Unexpected character: a
Unexpected character: a
Unexpected character: u
Unexpected character: u
Unexpected character: s
Unexpected character: s
Unexpected character: &
Unexpected character: &
Unexpected character: #
Unexpected character: #
Unexpected character: i
Unexpected character: i
Unexpected character: v
Unexpected character: v
Average loss at step 5300: 1.739849 learning rate: 1.000000
Minibatch perplexity: 5.53
Validation set perplexity: 5.65
Unexpected character: i
Unexpected character: i
Unexpected character: v
Unexpected character: v
Unexpected character: „
Unexpected character: „
Unexpected character: “
Unexpected character: “
Unexpected character: e
Unexpected character: e
Unexpected character: t
Unexpected character: t
Unexpected character: n
Unexpected character: n
Unexpected character: i
Unexpected character: i
Unexpected character: h
Unexpected character: h
Unexpected character: i
Unexpected character: i
Unexpected character: l
Unexpected charac

Unexpected character: “
Unexpected character: “
Unexpected character: a
Unexpected character: a
Unexpected character: v
Unexpected character: v
Average loss at step 5500: 1.720327 learning rate: 1.000000
Minibatch perplexity: 5.72
Validation set perplexity: 5.65
Unexpected character: №
Unexpected character: №
Unexpected character: &
Unexpected character: &
Unexpected character: #
Unexpected character: #
Unexpected character: a
Unexpected character: a
Unexpected character: s
Unexpected character: s
Unexpected character: s
Unexpected character: s
Unexpected character: e
Unexpected character: e
Unexpected character: z
Unexpected character: z
Unexpected character: c
Unexpected character: c
Unexpected character: a
Unexpected character: a
Unexpected character: u
Unexpected character: u
Unexpected character: s
Unexpected character: s
Unexpected character: &
Unexpected character: &
Unexpected character: #
Unexpected character: #
Unexpected character: 	
Unexpected character: 	
Unexpected charac

Unexpected character: m
Unexpected character: m
Unexpected character: a
Unexpected character: a
Unexpected character: d
Unexpected character: d
Unexpected character: a
Unexpected character: a
Unexpected character: m
Unexpected character: m
Unexpected character: e
Unexpected character: e
Unexpected character: v
Unexpected character: v
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: i
Unexpected character: –
Unexpected character: –
Unexpected character: v
Unexpected character: v
Unexpected character: i
Unexpected character: i
Unexpected character: 	
Unexpected character: 	
Unexpected character: 	
Unexpected character: 	
Unexpected character: d
Unexpected character: d
Unexpected character: u
Unexpected character: u
Unexpected character: h
Unexpected chara

Validation set perplexity: 5.59
Unexpected character: „
Unexpected character: „
Unexpected character: №
Unexpected character: №
Unexpected character: “
Unexpected character: “
Unexpected character: „
Unexpected character: „
Unexpected character: x
Unexpected character: x
Unexpected character: x
Unexpected character: x
Unexpected character: v
Unexpected character: v
Unexpected character: “
Unexpected character: “
Unexpected character: №
Unexpected character: №
Unexpected character: n
Unexpected character: n
Unexpected character: i
Unexpected character: i
Unexpected character: h
Unexpected character: h
Unexpected character: i
Unexpected character: i
Unexpected character: l
Unexpected character: l
Unexpected character: e
Unexpected character: e
Unexpected character: s
Unexpected character: s
Unexpected character: t
Unexpected character: t
Unexpected character: –
Unexpected character: –
Unexpected character: a
Unexpected character: a
Unexpected character: p
Unexpected character: p
Unexpect